<a href="https://colab.research.google.com/github/the-fur-lover-sketch/effective-invention/blob/main/Music_player_0930.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# --- 必要ライブラリ ---
import requests
import re
import ast
import math
import numpy as np
from scipy.io.wavfile import write
from IPython.display import Audio, display
from google.colab import files

# =========================
# 手順1: 変数読み取り
# =========================
def load_variables():
    url = "https://github.com/the-fur-lover-sketch/effective-invention/raw/refs/heads/main/Sheet5_Sheet-generator-test0929"
    resp = requests.get(url)
    text = resp.text

    # 変数部分を抽出
    variables_block = re.search(r"#---variables---#(.*?)#---score data---#", text, re.S).group(1)

    variables = {}
    for line in variables_block.splitlines():
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        if "=" in line:
            name, expr = line.split("=", 1)
            name = name.strip()
            expr = expr.strip()
            expr = expr.replace("^", "**")  # Python形式に修正

            try:
                node = ast.parse(expr, mode="eval").body

                def eval_node(node):
                    if isinstance(node, ast.Constant):
                        return node.value
                    elif isinstance(node, ast.BinOp):
                        left = eval_node(node.left)
                        right = eval_node(node.right)
                        if isinstance(node.op, ast.Add):
                            return left + right
                        elif isinstance(node.op, ast.Sub):
                            return left - right
                        elif isinstance(node.op, ast.Mult):
                            return left * right
                        elif isinstance(node.op, ast.Div):
                            return left / right
                        elif isinstance(node.op, ast.Pow):
                            return left ** right
                    elif isinstance(node, ast.Name):
                        return variables[node.id]
                    else:
                        raise ValueError

                value = eval_node(node)
                variables[name] = value
            except Exception:
                # 数値以外は文字列のまま
                variables[name] = expr.strip("'")

    print("[INFO] 変数読み取り完了:", variables)
    return variables

# =========================
# 手順2: scoreデータ読み取り
# =========================
def split_score_line(line):
    parts = []
    buf = []
    stack = []
    in_single = False
    in_double = False
    i = 0
    while i < len(line):
        c = line[i]
        if c == "'" and not in_double:
            if i > 0 and line[i-1] == "\\":
                buf.append(c)
            else:
                in_single = not in_single
                buf.append(c)
        elif c == '"' and not in_single:
            if i > 0 and line[i-1] == "\\":
                buf.append(c)
            else:
                in_double = not in_double
                buf.append(c)
        elif not in_single and not in_double:
            if c in "([{":
                stack.append(c)
                buf.append(c)
            elif c in ")]}":
                if stack:
                    stack.pop()
                buf.append(c)
            elif c == ',' and not stack:
                parts.append(''.join(buf).strip())
                buf = []
            else:
                buf.append(c)
        else:
            buf.append(c)
        i += 1
    parts.append(''.join(buf).strip())
    return parts

def load_and_parse_score(url):
    resp = requests.get(url)
    text = resp.text

    score_block_match = re.search(r"#---score data---#(.*)", text, re.S)
    if not score_block_match:
        raise RuntimeError("score block not found in file")

    score_block = score_block_match.group(1)
    score_lines = [ln.strip() for ln in score_block.splitlines() if ln.strip()]

    parsed_score = []
    for idx, line in enumerate(score_lines, start=1):
        parts = split_score_line(line)
        if len(parts) != 4:
            print(f"[WARN] {idx}行目: フィールド数={len(parts)} (期待:4)")
        else:
            print(f"[DEBUG] {idx}行目 -> {parts}")
        parsed_score.append(parts)

    return parsed_score

# =========================
# 手順3: 評価
# =========================
def safe_eval(expr, local_vars):
    expr = expr.replace("^", "**")
    try:
        return eval(expr, {"__builtins__": None, "math": math}, local_vars)
    except Exception as e:
        print("[ERROR] eval failed:", expr, e)
        return expr

def evaluate_score(parsed_score, vars_dict):
    evaluated_data = []
    for row_idx, row in enumerate(parsed_score, start=1):
        new_row = []
        print(f"\n--- Evaluating row {row_idx} ---")
        for i, field in enumerate(row):
            if i == 1 and field.startswith("f_"):
                var_name, list_str = field.split("*", 1)
                var_val = vars_dict[var_name]
                lst = ast.literal_eval(list_str)
                evaluated_list = [safe_eval(f"{var_val}*({x})", vars_dict) for x in lst]
                print(f"[DEBUG] field{i+1}: {field} -> {evaluated_list}")
                new_row.append(evaluated_list)
            else:
                val = safe_eval(field, vars_dict)
                print(f"[DEBUG] field{i+1}: {field} -> {val}")
                new_row.append(val)
        evaluated_data.append(new_row)
    return evaluated_data

# =========================
# WAV生成
# =========================
def generate_wav(evaluated_data, filename='output_debug.wav', fs=44100):
    # 長さ計算
    total_time = max(row[0]+row[2] for row in evaluated_data)
    print(f"[INFO] 総再生時間: {total_time}s")
    audio = np.zeros(int(fs * (total_time + 1)))

    for row_idx, row in enumerate(evaluated_data, start=1):
        start, freqs, duration, waveform = row
        t = np.arange(0, duration, 1/fs)
        sig = np.zeros_like(t)
        for f in freqs:
            if waveform == 'sine':
                sig += np.sin(2 * np.pi * f * t)
            # 他の波形も追加可能
        sig /= len(freqs)  # 複数周波数平均
        start_idx = int(start*fs)
        audio[start_idx:start_idx+len(sig)] += sig
        print(f"[DEBUG] row {row_idx}: start={start}s, duration={duration}s, waveform={waveform}, freqs={freqs}")

    # 正規化
    audio /= np.max(np.abs(audio))
    write(filename, fs, (audio*32767).astype(np.int16))
    print(f"[INFO] WAVファイル生成完了: {filename}")
    return filename

# =========================
# 実行フロー
# =========================
url = "https://github.com/the-fur-lover-sketch/effective-invention/raw/refs/heads/main/Sheet5_Sheet-generator-test0929"
vars_dict = load_variables()
parsed_score = load_and_parse_score(url)
evaluated_data = evaluate_score(parsed_score, vars_dict)
wav_file = generate_wav(evaluated_data)

# --- 再生 ---
display(Audio(wav_file, autoplay=False))

# --- ダウンロード ---
files.download(wav_file)


[INFO] 変数読み取り完了: {'t_1': 3, 't_2': 0.125, 'sp_1': 0, 'f_1': 110, 'f_2': 165, 'r': 1.0594630943592953, 'dr_1': 0.6, 'da_1': 0, 'wave_1': 'sine'}
[DEBUG] 1行目 -> ['t_1*1 + sp_1', "f_1*['4', '5', '6']", 'dr_1*t_1 + da_1', 'wave_1']
[DEBUG] 2行目 -> ['t_1*2 + sp_1', "f_1*['4', '4*r^4', '4*r^7']", 'dr_1*t_1 + da_1', 'wave_1']
[DEBUG] 3行目 -> ['t_2*50 + sp_1', "f_2*['2']", 'dr_1*t_2 + da_1', 'wave_1']
[DEBUG] 4行目 -> ['t_2*51 + sp_1', "f_2*['4']", 'dr_1*t_2 + da_1', 'wave_1']
[DEBUG] 5行目 -> ['t_2*52 + sp_1', "f_2*['2']", 'dr_1*t_2 + da_1', 'wave_1']
[DEBUG] 6行目 -> ['t_2*53 + sp_1', "f_2*['2']", 'dr_1*t_2 + da_1', 'wave_1']
[DEBUG] 7行目 -> ['t_2*54 + sp_1', "f_2*['4']", 'dr_1*t_2 + da_1', 'wave_1']
[DEBUG] 8行目 -> ['t_2*55 + sp_1', "f_2*['2']", 'dr_1*t_2 + da_1', 'wave_1']
[DEBUG] 9行目 -> ['t_2*56 + sp_1', "f_2*['4']", 'dr_1*t_2 + da_1', 'wave_1']
[DEBUG] 10行目 -> ['t_2*57 + sp_1', "f_2*['4']", 'dr_1*t_2 + da_1', 'wave_1']
[DEBUG] 11行目 -> ['t_2*58 + sp_1', "f_1*['2']", 'dr_1*t_2 + da_1', 'wave_1']
[DE

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>